In [2]:
import pandas as pd

# Read the CSV file with a specified encoding
df = pd.read_csv("cleaned_flight_data.csv", encoding="ISO-8859-1")

# Display the first 5 rows
df.head()

C:\Users\elena\AppData\Local\Temp\ipykernel_32856\1520107860.py:4: DtypeWarning: Columns (69,76,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("cleaned_flight_data.csv", encoding="ISO-8859-1")


,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum
0,1998,1,1,2,5,1998-01-02,NW,19386,NW,N297US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009,2,5,28,4,2009-05-28,FL,20437,FL,N946AT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,2,6,29,6,2013-06-29,MQ,20398,MQ,N665MQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,3,8,31,2,2010-08-31,DL,19790,DL,N6705Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2006,1,1,15,7,2006-01-15,US,20355,US,N504AU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [4]:
# --- Preprocessing ---
# Fill missing values for numerical columns
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].median())

In [5]:
# Create binary target variable 'DelayCategory': 1 if there was a delay, 0 if there wasn't
df['DelayCategory'] = df.apply(lambda row: 1 if row['DepDelay'] > 0 or row['ArrDelay'] > 0 else 0, axis=1)

In [6]:
# Encode 'DelayCategory'
le = LabelEncoder()
df['DelayCategory'] = le.fit_transform(df['DelayCategory'])

In [7]:
# Feature engineering (time-related features)
df['DepTime_combined'] = df['DepTime'] // 100 * 60 + df['DepTime'] % 100
df['ArrTime_combined'] = df['ArrTime'] // 100 * 60 + df['ArrTime'] % 100

# Seasonal and time-of-day features
df['HighDelaySeason'] = df['Month'].apply(lambda x: 1 if x in [6, 7, 12] else 0)
df['LateNightFlight'] = df['DepTime_combined'].apply(lambda x: 1 if 180 <= x % 1440 <= 240 else 0)
df['LateArrivalFlight'] = df['ArrTime_combined'].apply(lambda x: 1 if 180 <= x % 1440 <= 240 else 0)

# Delay-related features
df['CarrierDelay'] = df['DepDelay'].apply(lambda x: 1 if x > 0 else 0)
df['WeatherDelay'] = df['ArrDelay'].apply(lambda x: 1 if x > 0 else 0)
df['ArrDelayFlag'] = df['ArrDelay'].apply(lambda x: 1 if x > 0 else 0)

df['WeatherDelayHigh'] = df['WeatherDelay'].apply(lambda x: 1 if x > 1250 else 0)
df['CarrierDelayMid'] = df['CarrierDelay'].apply(lambda x: 1 if 0 < x <= 250 else 0)

In [8]:
# Feature selection
features = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime_combined', 'ArrTime_combined', 
            'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Flights', 'Distance', 
            'HighDelaySeason', 'LateNightFlight', 'LateArrivalFlight', 'WeatherDelayHigh', 'CarrierDelayMid']

In [9]:
# Prepare feature matrix and target vector
X = df[features]
y = df['DelayCategory']

In [10]:
# Label encode any object/categorical columns (just in case)
label_encoder = LabelEncoder()
for col in X.select_dtypes(include=['object']).columns:
    X[col] = label_encoder.fit_transform(X[col].astype(str))

# Fill any remaining missing values
X = X.fillna(X.median())

# --- Standardization (important for Logistic Regression) ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [11]:
# --- Train-test split ---
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [12]:
# --- Logistic Regression Model ---
logreg = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
logreg.fit(X_train, y_train)

# --- Predictions and Evaluation ---
y_pred = logreg.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))
print ("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

print("Accuracy Score:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.97      0.94    187866
           1       0.98      0.90      0.94    212134

    accuracy                           0.94    400000
   macro avg       0.94      0.94      0.94    400000
weighted avg       0.94      0.94      0.94    400000

Confusion Matrix: 
[[183032   4834]
 [ 20468 191666]]
Accuracy Score: 0.936745


### XGBOOST 

In [ ]:
# pip install xgboost scikit-learn pandas

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 1.3/150.0 MB 5.2 MB/s eta 0:00:29
    --------------------------------------- 2.6/150.0 MB 5.6 MB/s eta 0:00:27
    --------------------------------------- 3.7/150.0 MB 5.5 MB/s eta 0:00:27
   - -------------------------------------- 4.7/150.0 MB 5.7 MB/s eta 0:00:26
   - -------------------------------------- 6.0/150.0 MB 5.5 MB/s eta 0:00:27
   - -------------------------------------- 7.3/150.0 MB 5.7 MB/s eta 0:00:26
   -- ------------------------------------- 8.4/150.0 MB 5.7 MB/s eta 0:00:25
   -- ------------------------------------- 9.4/150.0 MB 5.6 MB/s eta 0:00:25
   -- ------------------------------------- 10.7/150.0 MB 5.6 MB/s eta 0:00:25
   --- ----

In [14]:
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix

In [15]:
# XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

C:\Users\elena\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [23:31:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [16]:
# Predict and evaluate
y_pred_xgb = xgb_model.predict(X_test)
print("\n--- XGBoost Classification Report ---")
print(classification_report(y_test, y_pred_xgb))
print("XGBoost Accuracy Score:", accuracy_score(y_test, y_pred_xgb))                                                                     
print("XGBoost Confusion Matrix:\n", confusion_matrix(y_test, y_pred_xgb))


--- XGBoost Classification Report ---
              precision    recall  f1-score   support

           0       0.97      0.97      0.97    187866
           1       0.97      0.97      0.97    212134

    accuracy                           0.97    400000
   macro avg       0.97      0.97      0.97    400000
weighted avg       0.97      0.97      0.97    400000

XGBoost Accuracy Score: 0.9681425
XGBoost Confusion Matrix:
 [[181438   6428]
 [  6315 205819]]


### LIGHTGBM

In [ ]:
# pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [19]:
from lightgbm import LGBMClassifier

In [20]:
# LightGBM model
lgbm_model = LGBMClassifier(random_state=42)
lgbm_model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 848341, number of negative: 751659
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.044590 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1635
[LightGBM] [Info] Number of data points in the train set: 1600000, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.530213 -> initscore=0.121000
[LightGBM] [Info] Start training from score 0.121000


LGBMClassifier(random_state=42)

In [21]:
# Predict and evaluate
y_pred_lgbm = lgbm_model.predict(X_test)
print("\n--- LightGBM Classification Report ---")
print(classification_report(y_test, y_pred_lgbm))
print("LightGBM Accuracy Score:", accuracy_score(y_test, y_pred_lgbm))

C:\Users\elena\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



--- LightGBM Classification Report ---
              precision    recall  f1-score   support

           0       0.96      0.97      0.97    187866
           1       0.97      0.96      0.97    212134

    accuracy                           0.97    400000
   macro avg       0.97      0.97      0.97    400000
weighted avg       0.97      0.97      0.97    400000

LightGBM Accuracy Score: 0.9674425


In [22]:
print("LightGBM Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lgbm))

LightGBM Confusion Matrix:
 [[182288   5578]
 [  7445 204689]]
